Running the WMMSE algorithm for the following setup:

- MIMO interference channel
- Noise powers and weights are set equally for all users
- P = 10^(SNR/10); SNR = 25 dB
- 4 cells
- n_tx = all 3
- n_rx = all 2
- Channel coefficients generated from CN(0, 1)


In [10]:
import os
import sys
import json
import torch
import importlib
# Get the current working directory
scripts_dir = os.getcwd()
# Go up one level
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)

import src.WMMSE
importlib.reload(src.WMMSE)
from src.WMMSE import WMMSE_alg

import src.utils
importlib.reload(src.utils)
from src.utils import calculate_sum_rate

In [ ]:
K = 3  # number of cells
I_k = [2, 3, 1]  # number of users in each cell
n_tx = [4, 2, 3]  # number of antennas at each transmitter
n_rx = [[2, 3], [1, 2, 2], [4]]  # number of antennas at each user in each cell
P_k = [5, 6, 7]
sig_i_k = [[.2, .3], [.1, .2, .2], [.4]]
d = [[5, 5], [5, 5, 5], [5]]
alpha = [[1, 1], [1, 1, 1], [1]]

# Initialize channel dictionary
H = {}
for k in range(K):  # transmitter cell index
    H[k] = {}
    for l in range(K):  # receiver cell index
        for i in range(I_k[l]):  # user index in cell l
            tx_ant = n_tx[k]
            rx_ant = n_rx[l][i]
            # Channel from transmitter k to user (l, i)
            H[k][(l, i)] = torch.randn(rx_ant, tx_ant, dtype=torch.cfloat)

max_iter_mu = 1000
tol_mu = 1e-3
max_iter_alg = 1000
tol_alg = 1e-3

In [7]:
wmmse = WMMSE_alg(K, I_k, n_tx, n_rx, H, P_k, sig_i_k, d, alpha, max_iter_mu, tol_mu, max_iter_alg, tol_alg)
V = wmmse.algorithm()

In [4]:
SNR = 25
P = 10**(SNR/10)
K = 4  # number of cells
I_k = [1, 1, 1, 1]  # number of users in each cell
n_tx = [3, 3, 3, 3]  # number of antennas at each transmitter
n_rx = [[2], [2], [2], [2]]  # number of antennas at each user in each cell
P_k = [P, P, P, P]
sig_i_k = [[1], [1], [1], [1]]
d = [[2], [2], [2], [2]]
alpha = [[1], [1], [1], [1]]

# Initialize channel dictionary
H = {}
for k in range(K):  # transmitter cell index
    H[k] = {}
    for l in range(K):  # receiver cell index
        for i in range(I_k[l]):  # user index in cell l
            tx_ant = n_tx[k]
            rx_ant = n_rx[l][i]
            # Channel from transmitter k to user (l, i)
            H[k][(l, i)] = torch.randn(rx_ant, tx_ant)/(2**.5) + 1j*torch.randn(rx_ant, tx_ant)/(2**.5)

max_iter_mu = 1000
tol_mu = 1e-3
max_iter_alg = 1000
tol_alg = 1e-3

In [8]:
wmmse = WMMSE_alg(K, I_k, n_tx, n_rx, H, P_k, sig_i_k, d, alpha, max_iter_mu, tol_mu, max_iter_alg, tol_alg)
V = wmmse.algorithm()

In [14]:
calculate_sum_rate(H, V, alpha, sig_i_k)

TypeError: unsupported operand type(s) for @: 'Tensor' and 'dict'

In [13]:
def calculate_sum_rate(H, V, alpha, sig):
    sum_rate = 0
    for k in range(len(H)):
        for i in range(len(H[k])):
            Nr = H[0][(k, i)].shape[0]
            S = torch.zeros((Nr, Nr))
            for j in range(len(H)):
                for l in range(len(H[k])):
                    if (j, l) == (k, i):
                        continue
                    S += H[j][(k, i)] @ V[j][l] @ V[j][l].conj().T @ H[j][(k, i)].conj().T
            S += sig[k][i] * torch.eye(Nr)
            tmp = torch.eye(Nr) + H[k][(k, i)] @ V[k][i] @ V[k][i].conj().T @ H[k][(k, i)].conj().T @ torch.linalg.inv(S)
            R = torch.log2(torch.linalg.det(tmp))
            sum_rate += alpha[k][i] * R
    return sum_rate

In [15]:
V

({0: {0: tensor([[5.7280e-05-8.2528e-05j, 8.8548e-05-2.9741e-04j],
           [9.6213e-05+1.0969e-05j, 4.3275e-04+1.7815e-04j],
           [8.1502e-06-7.5715e-05j, 6.9004e-05-2.6906e-04j]])},
  1: {0: tensor([[-0.0012-0.0003j, -0.0011+0.0011j],
           [-0.0002-0.0030j, -0.0033-0.0016j],
           [ 0.0008-0.0008j, -0.0003-0.0014j]])},
  2: {0: tensor([[ 0.0032-1.6591e-03j,  0.0007-3.9136e-03j],
           [-0.0009-8.7580e-04j, -0.0014-7.7557e-05j],
           [ 0.0018-3.2614e-03j, -0.0014-3.7337e-03j]])},
  3: {0: tensor([[-0.0014-2.5171e-04j,  0.0012+1.4096e-06j],
           [-0.0050+3.1080e-03j,  0.0038-3.2749e-03j],
           [-0.0025-2.1089e-04j,  0.0022-1.8360e-04j]])}},
 {0: {0: tensor([[ 2.7551e-05-4.3350e-05j,  8.5055e-05-1.2296e-04j],
           [-9.8579e-07+8.5531e-07j, -1.0607e-04+8.0654e-06j]])},
  1: {0: tensor([[-0.0003+0.0004j,  0.0002+0.0006j],
           [ 0.0004+0.0002j,  0.0004-0.0004j]])},
  2: {0: tensor([[ 0.0004+0.0008j,  0.0008+0.0002j],
           [ 0.000